In [ ]:
#verrijking van de harstilstand data doen eerst, dat is het allereerste. dan de slechte harstilstanden eruit halen, die dat al 
#volledig gedekt zijn door de ziekenhuizen, want die hoeven toch al niet meer gered te worden. Dan kan je zien welke AED's nuttig zijn en welke niet,
#door die netwerk analyse toe te passen en te zien welke AED's niet verbonden zijn (niet de dichtstbijzijnde) bij een harstilstand zijn. (PAS OP VOOR
#POTENTIELE RANDGEVALLEN, ALS ER BIJVOORBEELD meerdere harstilstanden vlak naast elkaar zijn, kan het zijn dat een AED eigenlijk slecht is maar toch het
#dichtst staat bij een harstilstand, die al gedekt is => HIER REKENING MEE HOUDEN, => OPLOSSING GEEN BIPARTITE GRAPH GEBRUIKEN, JE VERBINDT ALLE 
#HARTSTILSTANDEN ONDER EEN BEPAALDE TIJD MET ALLE AED'S DIE ONDER DIE BEPAALDE TIJD ZIJN, EN DAN ACHTERAF KAN JE BESLISSEN HOE HET ZIT)

In [ ]:
#AED's die te dicht bij de ziekenhuizen staan en daarom verwijderd worden, moeten ook gemarkeerd worden om die reden (reason for
#removal: too close to hospital). Dus pas op, je moet dat ook nog apart doen, als je enkel de harstilstanden verwijdert en niet ook de AED's,
#dan lijkt het alsof de AED's die te dicht stonden ook verwijderd werden omdat ze geen harstilstand coverden, terwijl dat eignelijk niet de reden was.

In [10]:
import pandas as pd
aed_locations = pd.read_csv('aed_locations_processed.csv')
card_arrest = pd.read_csv('card_arrest_processed.csv')
hospital_coordinates = pd.read_csv('hospital_coordinates_processed.csv')
removed_AED = pd.read_csv('removed_AED.csv')

In [4]:
#to calculate the distances, we use haversine formula, where we take the shortest distance between two points (a straight line).
#this is not always realistic (suppose an AED is placed on the corner of a building, then a straight line is impossible to take). Therefore we
#add 15% distance, to account for this.

,latitude,longitude
0,51.231381,4.415761
1,51.205787,4.412513
2,51.206556,4.470778
3,51.363865,4.985064
4,51.174354,4.420085
...,...,...
392,50.025633,5.359641
393,50.674803,5.633665
394,50.652888,5.578356
395,50.622523,5.636381


In [11]:
aed_locations

,id,address,number,postal_code,municipality,province
0,13.0,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut
1,70.0,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège
2,71.0,Place Saint - Lambert,NaN,4020.0,Liège,Liège
3,72.0,Rue du Doyard,NaN,4990.0,Lierneux,Liège
4,73.0,Fond Saint Servais,NaN,4000.0,Liège,Liège
...,...,...,...,...,...,...
14471,16660.0,Chaussée de Marche,799.0,5100.0,Wierde,Namur
14472,16661.0,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen
14473,16664.0,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen
14474,16665.0,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen


In [8]:
aed_locations_test = pd.read_csv('aed_locations_processed_test.csv')

In [9]:
aed_locations_test

,id,address,number,postal_code,municipality,province
0,13.0,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut
1,70.0,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège
2,71.0,Place Saint - Lambert,NaN,4020.0,Liège,Liège
3,72.0,Rue du Doyard,NaN,4990.0,Lierneux,Liège
4,73.0,Fond Saint Servais,NaN,4000.0,Liège,Liège
...,...,...,...,...,...,...
14471,16660.0,Chaussée de Marche,799.0,5100.0,Wierde,Namur
14472,16661.0,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen
14473,16664.0,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen
14474,16665.0,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen
